In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
import torch

In [3]:
device_name = torch.cuda.get_device_name(0)

torch_device_name = "cpu" if "AMD Radeon RX 580 2048SP" in device_name else "cuda"

print(f"Arquitetura de dispositivo: '{device_name}' e nome do dispositivo: '{torch_device_name}'" )

Arquitetura de dispositivo: 'AMD Radeon RX 580 2048SP' e nome do dispositivo: 'cpu'


In [56]:
class CPSolver:
    def __init__(self, model_path="t5-small-cp-solver-4", torch_device_name=torch_device_name):
        self.model = T5ForConditionalGeneration.from_pretrained(model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)
        self.device = torch.device(torch_device_name)
        self.model.to(self.device)

    def solve(self, problem_statement):
        # Preprocessamento do input para que o modelo possa entender melhor e gerar uma saída mais precisa.
        # Testamos vários tipos de strings de entrada, as menores tendem a criar saídas menos precisas.
        input_text = f"Generate the step-by-step programming for the problem: {problem_statement[:9000]}"

        inputs = self.tokenizer(
            input_text,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        ).to(self.device)

        outputs = self.model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=512,
            num_beams=4,  # Increased beam width
            early_stopping=False,
            length_penalty=2,  # Encourage shorter sequences
            no_repeat_ngram_size=2,  # Prevent 3-gram repeats
            temperature=0.8,  # Add some randomness
            top_k=50,               # Consider top 50 tokens
            top_p=0.95,             # Nucleus sampling
            repetition_penalty=2.0, # Explicit penalty
            num_return_sequences=2,  # Generate multiple candidates
            eos_token_id=self.tokenizer.eos_token_id,
            forced_eos_token_id=None  # Remove forced EOS

        )

        candidates = [self.tokenizer.decode(seq, skip_specials=True) for seq in outputs]
        return max(candidates, key=lambda x: len(x.split()))

In [57]:
solver = CPSolver(model_path='models/lighting4')

In [48]:
problems = [
  "You are given two positive integers and In one move you can increase by replace with Your task is to find the minimum number of moves you need to do in order to make divisible by It is possible that you have to make moves as is already divisible by You have to answer independent test cases",
  "You are given two positive integers and In one move you can in the following way Choose any positive integer and replace with choose any positive integer and replace with You can perform as many such operations as you want You can choose the same numbers and in different moves Your task is to find the minimum number of moves required to obtain from It is guaranteed that you can always obtain from You have to answer independent test cases",
  "Polycarp has positive integers and He can perform the following operation Choose a integer and multiply of the integers or by Can Polycarp make it so that after performing the operation the sequence of three numbers forms an arithmetic progression Note that you the order of and Formally a sequence is called an arithmetic progression AP if there exists a number called common difference such that for all from to In this problem For example the following sequences are AP and The following sequences are not AP and You need to answer independent test cases ",
  "You are given a string consisting of lowercase English letters You must print the last character of the string",
  "There are N pigeons numbered from 1 to N, and there are N nests numbered from 1 to N Initially, pigeon i is in nest i for 1 less than N You are given Q queries, which you must process in order. There are two types of queries, each given in one of the following formats: Move P pigeon to nest H, Output the number of nests that contain more than one pigeon."
]

In [61]:
problem = problems[0]
solution = solver.solve(problem)

In [36]:
import re

In [10]:
def format_str(s):
    return re.sub(r'(?=[A-Z])', '\n', s)

In [62]:
print("STATEMENT:")
print(format_str(problem))
print()
print("EDITORIAL GERADO:")
print(format_str(solution))

STATEMENT:

You are given two positive integers and 
In one move you can increase by replace with 
Your task is to find the minimum number of moves you need to do in order to make divisible by 
It is possible that you have to make moves as is already divisible by 
You have to answer independent test cases

EDITORIAL GERADO:
<pad> <extra_id_0> by 
Then it is possible that you can make moves as is already divisible by 
So if you want to solve the problem you need to find the minimum number of moves in order to make them 
Divisible 
By multiplying by two positive integers and replacing with one negative integer 
You can use this function to calculate the maximum amount of moving you have to do 
If you are going to increase by we will add to the equation and replace with 
Now let s look at the value of the answer for the first move and check whether or not all of these values are equal to and so on 
Let the code be the same from to 
It is easy to see that there are no more than two ways to